In [1]:
from adversarial_lab.core.noise_generators import AdditiveNoiseGenerator
from adversarial_lab.attacks.whitebox import WhiteBoxMisclassification
from adversarial_lab.core.preprocessing import PreprocessingFromFunction
from adversarial_lab.utils.plotting import plot_images_and_noise

from PIL import Image
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions

In [2]:
model = InceptionV3(weights='imagenet')

def preprocess(sample, *args, **kwargs):
    input = tf.cast(sample, dtype=tf.float32)
    if len(input.shape) == 2:
        input = tf.expand_dims(input, axis=-1)
        input = tf.image.grayscale_to_rgb(input)

    elif len(input.shape) == 3 and input.shape[-1] == 1:
        input = tf.image.grayscale_to_rgb(input)

    input_tensor = tf.convert_to_tensor(input, dtype=tf.float32)
    resized_image = tf.image.resize(input_tensor, (299, 299))
    batch_image = tf.expand_dims(resized_image, axis=0)
    return preprocess_input(batch_image)

pp = PreprocessingFromFunction.create(preprocess)

In [3]:
image = Image.open('data/panda.jpg')
image_array = np.array(image)

np.argmax(model.predict(pp.preprocess(image_array)), axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


array([388], dtype=int64)

In [4]:
attacker = WhiteBoxMisclassification(model, "cce", "adam", preprocessing=pp)
attacker.model.set_compute_jacobian(True)

In [ ]:
noise = attacker.attack(image_array, epochs=100, strategy="random", verbose=3)

Attacking:   1%|          | 1/100 [01:03<1:44:06, 63.09s/it, Loss=6.94, Prediction=388, Prediction Confidence=0.928]

In [ ]:
plot_images_and_noise(pp.preprocess(image_array), noise, attacker.noise_generator)

TypeError: plot_images_and_noise() takes 2 positional arguments but 3 were given